In [1]:
import base64
import requests
from datetime import datetime, timedelta
from urllib.parse import urlencode

In [2]:
client_id = 'ca305d1bc7e64485b99c494669317577'
client_secret = 'e00556e638e84fd99da29d554ae66778'

In [3]:
class SpotifyAPI:
    access_token = None
    access_token_expires = datetime.now()
    access_token_expired = True
    client_id = None
    client_secret = None
    token_url = 'https://accounts.spotify.com/api/token'
    
    def __init__(self, client_id, client_secret):
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_client_credentials(self):
        '''
        Returns a base64 encoded string
        '''
        client_id = self.client_id
        client_secret = self.client_secret
        
        if client_id == None or client_secret == None:
            raise Exception('You must set client_id and client_secret.')

        client_creds = f'{client_id}:{client_secret}'
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        return {'Authorization': f'Basic {self.get_client_credentials()}'}
    
    def get_token_data(self):
        return {'grant_type': 'client_credentials'}
    
    def auth(self):
        r = requests.post(self.token_url, data=self.get_token_data(), headers=self.get_token_headers())
        if r.status_code not in range(200, 300):
            return Exception('Could not authenticate client.')
        data = r.json()
        now = datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in']  # seconds
        expires = now + timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_expired = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.now()
        if expires < now:
            self.auth()
            return self.get_access_token()
        elif token == None:
            self.auth()
            return self.get_access_token()
        return token
    
    def get_resource_header(self):
        return {'Authorization': f'Bearer {self.get_access_token()}'}
    
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 300):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resource(lookup_id=_id, resource_type='albums')
    
    def get_artist(self, _id):
        return self.get_resource(lookup_id=_id, resource_type='artists')
    
    def search(self, query, search_type='artist'):
        headers = self.get_resource_header()
        endpoint = 'https://api.spotify.com/v1/search'
        data = urlencode({'q': query, 'type': search_type.lower()})

        lookup_url = f"{endpoint}?{data}"

        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 300):
            return {}
        return r.json()

In [4]:
spotify = SpotifyAPI(client_id, client_secret)

In [9]:
spotify.search(query='hallelujah', search_type='track')

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=hallelujah&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3nnQpaTvKb5jCQabZefACI'},
       'href': 'https://api.spotify.com/v1/artists/3nnQpaTvKb5jCQabZefACI',
       'id': '3nnQpaTvKb5jCQabZefACI',
       'name': 'Jeff Buckley',
       'type': 'artist',
       'uri': 'spotify:artist:3nnQpaTvKb5jCQabZefACI'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ',
      'DE',
      'DJ',
      'DK',
    

In [10]:
spotify.get_artist('3nnQpaTvKb5jCQabZefACI')

{'external_urls': {'spotify': 'https://open.spotify.com/artist/3nnQpaTvKb5jCQabZefACI'},
 'followers': {'href': None, 'total': 744267},
 'genres': ['art rock',
  'melancholia',
  'permanent wave',
  'rock',
  'singer-songwriter'],
 'href': 'https://api.spotify.com/v1/artists/3nnQpaTvKb5jCQabZefACI',
 'id': '3nnQpaTvKb5jCQabZefACI',
 'images': [{'height': 1520,
   'url': 'https://i.scdn.co/image/67779606c7f151618a28f62b1d24fb514d39dacf',
   'width': 1000},
  {'height': 973,
   'url': 'https://i.scdn.co/image/248e8ddbf1be229fd0c31722fb0a01b31b01f474',
   'width': 640},
  {'height': 304,
   'url': 'https://i.scdn.co/image/8822f816dea6ea25c70e942763c9812c6271e516',
   'width': 200},
  {'height': 97,
   'url': 'https://i.scdn.co/image/2a93d455e8981b0e1f6f1918b934c454c1243b20',
   'width': 64}],
 'name': 'Jeff Buckley',
 'popularity': 66,
 'type': 'artist',
 'uri': 'spotify:artist:3nnQpaTvKb5jCQabZefACI'}

In [11]:
spotify.get_album('7yQtjAjhtNi76KRu05XWFS')

{'album_type': 'album',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3nnQpaTvKb5jCQabZefACI'},
   'href': 'https://api.spotify.com/v1/artists/3nnQpaTvKb5jCQabZefACI',
   'id': '3nnQpaTvKb5jCQabZefACI',
   'name': 'Jeff Buckley',
   'type': 'artist',
   'uri': 'spotify:artist:3nnQpaTvKb5jCQabZefACI'}],
 'available_markets': ['AD',
  'AE',
  'AG',
  'AL',
  'AM',
  'AO',
  'AR',
  'AT',
  'AU',
  'AZ',
  'BA',
  'BB',
  'BD',
  'BE',
  'BF',
  'BG',
  'BH',
  'BI',
  'BJ',
  'BN',
  'BO',
  'BR',
  'BS',
  'BT',
  'BW',
  'BY',
  'BZ',
  'CA',
  'CH',
  'CI',
  'CL',
  'CM',
  'CO',
  'CR',
  'CV',
  'CW',
  'CY',
  'CZ',
  'DE',
  'DJ',
  'DK',
  'DM',
  'DO',
  'DZ',
  'EC',
  'EE',
  'EG',
  'ES',
  'FI',
  'FJ',
  'FM',
  'FR',
  'GA',
  'GB',
  'GD',
  'GE',
  'GH',
  'GM',
  'GN',
  'GQ',
  'GR',
  'GT',
  'GW',
  'GY',
  'HK',
  'HN',
  'HR',
  'HT',
  'HU',
  'ID',
  'IE',
  'IL',
  'IN',
  'IS',
  'IT',
  'JM',
  'JO',
  'JP',
  'KE',
  'KG',
  'KH